In [2]:
import pandas as pd
import numpy as np
import os
from random import uniform
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
import re


In [3]:
os.chdir('../')


In [4]:
from mc_hammer.mchammer import mchammer

In [5]:
def mc_hammer_test(x,null_method,cluster_method,k = None,eps = None,min_samples = None):
    mch = mchammer()
    mch.get_null_distributions(x,null_method,repeats = 100)
    return mch.get_q_scores(cluster_method,k = k,eps=eps,min_samples=min_samples)

In [6]:
def get_true_list(test_list,true_k_ind):
    test_list2 = test_list[:true_k_ind] + test_list[true_k_ind + 1:]
    if (test_list2.count(True) == 0) and (test_list[true_k_ind] == True):
        return True
    else:
        return False
def get_true_dict(res_list,true_k_ind):
    res_dict = {i:[j[i] for j in res_list] for i in res_list[0].keys()}
    res_dict = {k:get_true_list(v,true_k_ind) for k,v in res_dict.items()}
    return res_dict

In [7]:
def big_mc_hammer(
    x_list,
    null_method,
    cluster_method,
    k_list = [2,4,5],
    eps_list = [0.5],
    min_samples_list = [4,5,7],
    spef = False,
    true_k_ind = None
):
    if cluster_method == 'DBSCAN':
        true_list=[mc_hammer_test(
            l,
            null_method,
            cluster_method,
            k=None,
            eps = j,
            min_samples=i
        ) for l in x_list for i in min_samples_list for j in eps_list]
    else:
        if spef:
            true_list=[[mc_hammer_test(
                j,
                null_method,
                cluster_method,
                k = i,
                eps = None,
                min_samples = None
            ) for i in k_list] for j in x_list] 
            
            true_list = [get_true_dict(i,true_k_ind) for i in true_list]
        else:
            true_list=[mc_hammer_test(
                j,
                null_method,
                cluster_method,
                k = i,
                eps = None,
                min_samples = None
            ) for j in x_list for i in k_list]
        true_dict = {i:[true_list[j][i] for j in range(len(true_list))] for i in true_list[0].keys()}
        true_dict_res = {k:v.count(True)/len(v)*100 for k,v in true_dict.items()}
    return true_dict_res

In [8]:
x_list = [make_classification(
        n_samples = 100,
        n_features = 10,
        n_informative = 10,
        n_redundant = 0,
        n_classes = 2,
        n_clusters_per_class = 1,
        class_sep = 1,
        random_state = i
    )[0] for i in range(4)]
true_list=[[mc_hammer_test(
    j,
    'min_max',
    'K_Means',
    k = i,
    eps = None,
    min_samples = None
) for i in k_list] for j in x_list] 

NameError: name 'k_list' is not defined

In [33]:
true_k_ind = 0
res_list = true_list[0]
res_dict = {i:[j[i] for j in res_list] for i in res_list[0].keys()}

def get_true_list(test_list,true_k_ind):
    test_list2 = test_list[:true_k_ind] + test_list[true_k_ind + 1:]
    if (test_list2.count(True) == 0) and (test_list[true_k_ind] == True):
        return True
    else:
        return False
def get_true_dict(res_list,true_k_ind):
    res_dict = {i:[j[i] for j in res_list] for i in res_list[0].keys()}
    res_dict = {k:get_true_list(v,true_k_ind) for k,v in res_dict.items()}
    return res_dict
[get_true_dict(i,true_k_ind) for i in true_list]

[{'huberts_gamma': False,
  'norm_gamma': False,
  'sillhouette_euclidean': False,
  'sillhouette_cosine': False,
  'CH': False,
  'DB': False},
 {'huberts_gamma': False,
  'norm_gamma': False,
  'sillhouette_euclidean': False,
  'sillhouette_cosine': True,
  'CH': False,
  'DB': False},
 {'huberts_gamma': False,
  'norm_gamma': False,
  'sillhouette_euclidean': False,
  'sillhouette_cosine': False,
  'CH': False,
  'DB': False},
 {'huberts_gamma': False,
  'norm_gamma': False,
  'sillhouette_euclidean': False,
  'sillhouette_cosine': False,
  'CH': False,
  'DB': False}]

In [9]:
def test_blobs(k,noise,seperation,test_true,spef = False,pca = False):
    print(str(k) + str(noise) + str(seperation))
    x_list =[make_classification(
        n_samples = 100,
        n_features = 10,
        n_informative = int((10-10*noise)),
        n_redundant = int(10*noise),
        n_classes = k,
        n_clusters_per_class = 1,
        class_sep = seperation,
        random_state = i
    )[0] for i in range(100)]
    null_method_list = ['pca_trans','random_order','min_max']
    cluster_method_list = ['K_Means']
    if pca: 
        x_list = [PCA(n_components = 0.9).fit_transform(i) for i in x_list]
    if test_true:
        k_range = [k]
    else:
        k_range = [2,4,5]
    
    if spef: 
        true_k_ind = k_range.index(k)
    else:
        true_k_ind = None
    results_dict = {i +'_' + j:big_mc_hammer(
        x_list = x_list,
        null_method = i,
        cluster_method = j,
        k_list = k_range,
        spef = spef,
        true_k_ind = true_k_ind
    ) for i in null_method_list for j in cluster_method_list}
    
    results_df = pd.DataFrame(results_dict).T
    return(results_df)

In [6]:
null_method_list = ['pca_trans','random_order','min_max']
cluster_method_list = ['K_Means','DBSCAN','spectral_clustering']
x_list = [np.random.rand(100,3) for i in range(100)]
null_results_dict = {i +'_' + j:big_mc_hammer(
    x_list = x_list,
    null_method = i,
    cluster_method = j
) for i in null_method_list for j in cluster_method_list}

In [7]:
null_results_dict

{'pca_trans_K_Means': {'huberts_gamma': 0.0,
  'norm_gamma': 97.66666666666667,
  'sillhouette_euclidean': 55.00000000000001,
  'sillhouette_cosine': 3.0,
  'CH': 55.333333333333336,
  'DB': 39.666666666666664},
 'pca_trans_DBSCAN': {'huberts_gamma': 0.0,
  'norm_gamma': 0.0,
  'sillhouette_euclidean': 0.0,
  'sillhouette_cosine': 0.0,
  'CH': 0.0,
  'DB': 0.0},
 'pca_trans_spectral_clustering': {'huberts_gamma': 2.0,
  'norm_gamma': 88.0,
  'sillhouette_euclidean': 48.0,
  'sillhouette_cosine': 8.333333333333332,
  'CH': 55.00000000000001,
  'DB': 33.33333333333333},
 'random_order_K_Means': {'huberts_gamma': 4.333333333333334,
  'norm_gamma': 4.0,
  'sillhouette_euclidean': 5.666666666666666,
  'sillhouette_cosine': 3.6666666666666665,
  'CH': 3.6666666666666665,
  'DB': 4.0},
 'random_order_DBSCAN': {'huberts_gamma': 0.0,
  'norm_gamma': 0.0,
  'sillhouette_euclidean': 0.0,
  'sillhouette_cosine': 0.0,
  'CH': 0.0,
  'DB': 0.0},
 'random_order_spectral_clustering': {'huberts_gamma':

In [33]:
null_df = pd.DataFrame(null_results_dict).T
null_df.to_csv('data/processed/null_results.csv')

In [10]:
k_list = [2,4,5]
noise_list = [0.1,0.3,0.5]
sep_list = [0.1,0.5,1]

In [28]:
blob_test = {'k-' + str(i) +'_noise-' +str(j) + '_sep-' + str(l):test_blobs(
    k = i,
    noise = j,
    seperation = l,
    test_true = True
) for i in k_list for j in noise_list for l in sep_list}

20.10.1
20.10.5
20.11
20.30.1
20.30.5
20.31
20.50.1
20.50.5
20.51
40.10.1
40.10.5
40.11
40.30.1
40.30.5
40.31
40.50.1
40.50.5
40.51
50.10.1
50.10.5
50.11
50.30.1
50.30.5
50.31
50.50.1
50.50.5
50.51


In [29]:
blob_test

{'k-2_noise-0.1_sep-0.1':                       huberts_gamma  norm_gamma  sillhouette_euclidean  \
 pca_trans_K_Means               0.0         2.0                    3.0   
 random_order_K_Means           53.0         2.0                  100.0   
 min_max_K_Means                 0.0         6.0                   63.0   
 
                       sillhouette_cosine     CH     DB  
 pca_trans_K_Means                    0.0    0.0    2.0  
 random_order_K_Means                98.0  100.0  100.0  
 min_max_K_Means                     62.0   65.0   76.0  ,
 'k-2_noise-0.1_sep-0.5':                       huberts_gamma  norm_gamma  sillhouette_euclidean  \
 pca_trans_K_Means               0.0         2.0                    2.0   
 random_order_K_Means           53.0         2.0                  100.0   
 min_max_K_Means                 0.0         7.0                   67.0   
 
                       sillhouette_cosine     CH     DB  
 pca_trans_K_Means                    0.0    0.0    1.0

In [ ]:
if os.path.isdir('data/processed/k_means_pos_test') == False:
    os.mkdir('data/processed/k_means_pos_test')
for k,v in blob_test.items():
    file_name = re.sub('\.','',k)
    v.to_csv('data/processed/k_means_pos_test/' + file_name +'.csv')

In [ ]:
blob_test_sens = {'k-' + str(i) +'_noise-' +str(j) + '_sep-' + str(l):test_blobs(
    k = i,
    noise = j,
    seperation = l,
    test_true = False,
    spef = True
) for i in k_list for j in noise_list for l in sep_list}

20.10.1
20.10.5
20.11
20.30.1
20.30.5
20.31
20.50.1
20.50.5
20.51


In [ ]:
if os.path.isdir('data/processed/k_means_sens_test') == False:
    os.mkdir('data/processed/k_means_sens_test')
for k,v in blob_test_sens.items():
    file_name = re.sub('\.','',k)
    v.to_csv('data/processed/k_means_sens_test/' + file_name +'.csv')

In [ ]:
blob_test_pca = {'k-' + str(i) +'_noise-' +str(j) + '_sep-' + str(l):test_blobs(
    k = i,
    noise = j,
    seperation = l,
    test_true = False,
    spef = True,
    pca = True
) for i in k_list for j in noise_list for l in sep_list}